In [9]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
np.seterr(divide='ignore')
lista_filtrada = []
#lista=md.filtradodemonedas ()
#lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ZRXUSDT', 'DOGEUSDT', 'DOTUSDT', 'YFIUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'AVAXUSDT', 'FTMUSDT', 'NEARUSDT', 'FILUSDT', 'MATICUSDT', 'SANDUSDT', 'DYDXUSDT', 'GALAUSDT', 'KLAYUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 'APTUSDT', 'ARBUSDT', 'SUIUSDT', 'BNTUSDT', 'SEIUSDT', 'HIFIUSDT', 'ARKUSDT', 'LOOMUSDT', 'GASUSDT', 'TIAUSDT', 'MEMEUSDT', 'ORDIUSDT']
lista = ['OPUSDT']

timeframe = '1h'
if len(lista) > 1:
    imprimo = False
    debug = False
else:
    imprimo = True
    debug = True
print(lista)
porcentajes_sumados=0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
balance = md.balancetotal()
for symbol in lista:    
    try:
        data = md.estrategia_smart(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe, balance = balance, largo = 1)
        #data = md.estrategia_alex(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe, balance = balance, largo = 10)
        resultado = md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if not np.isnan(resultado['Win Rate [%]']):
            if resultado['Win Rate [%]'] >= 50:
                win_rate_buenos = win_rate_buenos+1
            else:
                win_rate_malos = win_rate_malos+1
        if ((data.Close.iloc[-1] < (data.decisional_alcista_high.iloc[-2] + (data.atr.iloc[-1]*3)))
            or 
            (data.Close.iloc[-1] > (data.decisional_bajista_low.iloc[-2] - (data.atr.iloc[-1]*3)))
            ):
            print(f"posible entrada symbol: {symbol}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"\nTimeframe {timeframe} - porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos} - Ganancia por trade: {md.truncate((porcentajes_sumados/trades if trades !=0 else porcentajes_sumados),2)}%")


['OPUSDT']


C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


OPUSDT - Return [%]: 0.0% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 0.0
posible entrada symbol: OPUSDT

Timeframe 1h - porcentajes_sumados 0.0 - trades 1 - win_rate_buenos 0 - win_rate_malos 1 - Ganancia por trade: 0.0%


In [10]:
data

,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,...,tendencia,trend,alex,decisional_alcista_cerca,decisional_bajista_cerca,signal,take_profit,stop_loss,cierra,porcentajeentrada
2023-10-03 10:00:00,2023-10-03 10:00:00,1.3685,1.3695,1.3614,1.3669,3853645.4,NaN,NaN,NaN,NaN,...,-2.0,Alcista,-2,NaN,NaN,0,0.000000,0.000000,False,0.000000
2023-10-03 11:00:00,2023-10-03 11:00:00,1.3668,1.3669,1.3500,1.3543,6425003.5,NaN,NaN,NaN,NaN,...,-2.0,Alcista,-2,NaN,NaN,0,0.000000,0.000000,False,0.000000
2023-10-03 12:00:00,2023-10-03 12:00:00,1.3544,1.3582,1.3444,1.3529,5585097.6,NaN,NaN,NaN,NaN,...,-2.0,Alcista,-2,NaN,NaN,0,0.000000,0.000000,False,0.000000
2023-10-03 13:00:00,2023-10-03 13:00:00,1.3528,1.3639,1.3479,1.3565,4478082.9,NaN,NaN,NaN,NaN,...,-2.0,Alcista,-2,NaN,NaN,0,0.000000,0.000000,False,0.000000
2023-10-03 14:00:00,2023-10-03 14:00:00,1.3564,1.3564,1.3353,1.3449,10076594.9,NaN,NaN,NaN,NaN,...,-2.0,Alcista,-2,NaN,NaN,0,0.000000,0.000000,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13 21:00:00,2023-11-13 21:00:00,1.8517,1.8526,1.7684,1.8058,19247438.0,1.843887,1.805876,1.665234,0.047462,...,-1.0,Alcista,-2,0.0,0.0,0,0.000000,0.000000,False,0.000000
2023-11-13 22:00:00,2023-11-13 22:00:00,1.8060,1.8221,1.7880,1.7906,4166766.1,1.838812,1.805277,1.666481,0.046508,...,-1.0,Alcista,-2,0.0,0.0,0,0.000000,0.000000,False,0.000000
2023-11-13 23:00:00,2023-11-13 23:00:00,1.7907,1.8220,1.7880,1.7914,4144521.7,1.834297,1.804733,1.667724,0.045614,...,-1.0,Alcista,-3,0.0,0.0,0,0.000000,0.000000,False,0.000000
2023-11-14 00:00:00,2023-11-14 00:00:00,1.7914,1.7956,1.7423,1.7582,14468609.5,1.827050,1.802908,1.668624,0.046163,...,-1.0,Alcista,-2,0.0,0.0,1,2.203931,1.682012,False,47.723605
